# Introduction - RAG Chatbot with Semantic Kernel

### Implements a simple workflow of retrieve then generate. using `semantic kernel` to help with prompt engineering and orchestrating OpenAI API calls and Weaviate as knowledgebase from which to retreive semantically relevant context. This allows us to not only control what is being generated but also cite sources.

# Setup

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.18.230725.3-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Weaviate, 0.18.230725.3-preview"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.Weaviate;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Orchestration;

## OS-specific notes:
* if you run into SSL errors when connecting to OpenAI on macOS, see this issue for a [potential solution](https://github.com/microsoft/semantic-kernel/issues/627#issuecomment-1580912248)
* on Windows, you may need to run Docker Desktop as administrator

In [ ]:
var apiKey = "my-secret-key";
WeaviateMemoryStore memoryStore = new("http://localhost:8080/v1/", apiKey);

Then, we register the memory store to the kernel:

In [ ]:
var openApiKey = "";
IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder
                       .WithOpenAITextCompletionService("text-davinci-003", openApiKey)
                       .WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", openApiKey)
                       .WithMemoryStorage(memoryStore)
                       .Build();
kernel.ImportSkill(new TextMemorySkill(kernel.Memory));

# Manually adding memories


Let's create some initial memories "About Me". We can add memories to our weaviate memory store by using `save_information_async`

In [ ]:
string collectionName = "AboutMe";

In [ ]:
// Function for persisting a memory to Weaviate

async Task Remember(string memory, string collection, IKernel kernel)
{
    // Add some documents to the semantic memory
    await kernel.Memory.SaveInformationAsync(collection, memory, Guid.NewGuid().ToString());
}

In [ ]:
await Remember("When I turned 5 my parents gifted me goldfish for my birthday", collectionName,  kernel);

In [ ]:
await Remember("I love datascience", collectionName, kernel);

In [ ]:
await Remember("I have a black nissan sentra", collectionName, kernel);

In [ ]:
await Remember("my favourite food is popcorn", collectionName, kernel);

In [ ]:
await Remember("I like to take long walks.", collectionName, kernel);

In [ ]:
var result = kernel.Memory.SearchAsync(collectionName, "Do I have a pet?", limit: 1);
await foreach (var item in result)
{
    Console.WriteLine(item.Metadata.Text);
}

In [ ]:
var result2 = kernel.Memory.SearchAsync(collectionName, "passion", limit: 3);
await foreach (var item in result2)
{
    Console.WriteLine(item.Metadata.Text);
    Console.WriteLine(item.Relevance);
}

In [ ]:
async Task<Tuple<ISKFunction, SKContext>> SetupRAG(IKernel kernel)
{
    var prompt = @"""
            You are a friendly and talkative AI.
            Answer to the user question: {{$user_input}}
            You can, but don't have to, use relevant information provided here: {{$retreived_context}} 
            If you are not sure of the answer say ""I am not sure.""
            """.Trim();
    var func = kernel.CreateSemanticFunction(prompt);
    var context = kernel.CreateNewContext();
    context["chat_history"] = "";
    context["retreived_context"] = "";
    return Tuple.Create(func, context);
}

In [ ]:
using Microsoft.DotNet.Interactive;

async Task<bool> RAG(IKernel kernel, ISKFunction ragFunc, SKContext context)
{
    var userInput = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter text:");

    context["user_input"] = userInput;

    if (userInput == "exit") {
        Console.WriteLine("Exiting chat...");
        return false;
    }

    var result = kernel.Memory.SearchAsync(collectionName, userInput, limit: 5, minRelevanceScore: 0.5);
    var retrieved = new List<string>();
    await foreach (var item in result)
    {
        retrieved.Add(item.Metadata.Text);
    }
    context["retreived_context"] = string.Join(". ", retrieved).Trim();

    var answer = await ragFunc.InvokeAsync(context);

    // TODO: Insert the response back into Weaviate using remember()
    // Chat history is kind of bad, we might just want to retrieve 2-3 memories from Weaviate and use them so that 
    // input length doesn't grow as we keep chatting.

    context["chat_history"] += $"\nUser:> {userInput}\nChatBot:> {answer}\n";
    Console.WriteLine($"{answer}");

    return true;
}

In [ ]:
Console.WriteLine("Setting up a chat (with memory!)");
var (rag_func, context) = await SetupRAG(kernel);

Console.WriteLine("Begin chatting (type 'exit' to exit):\n");
var chatting = true;
while (chatting)
     chatting = await RAG(kernel, rag_func, context);

In [ ]:
// The chathistory can be obtained from the context.
Console.WriteLine(context.Variables["chat_history"]);